In [ ]:
import os, sys
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
sys.path.append('..')
import avstack
import avapi
import numpy as np
import cv2
from tqdm import tqdm

data_dir = '/data/spencer/CARLA/multi-agent-v1'
CSM = avapi.carla.CarlaScenesManager(data_dir, verbose=False)
CSD = CSM.get_scene_dataset_by_name(CSM.scenes[-2])

## Make Visualization

In [ ]:
frame_idx = 100
sens = "CAM_INFRASTRUCTURE_001"
frame = CSD.get_frames(sens)[frame_idx]
img = CSD.get_image(frame, sens)
# objs = [CSD.get_objects(frame, "CAM_FRONT")[0]]
objs = CSD.get_objects(frame, sens)
avapi.visualize.snapshot.show_image_with_boxes(img, objs, inline=True)

In [ ]:
i_frame_start = 6
n_safety_frames = 10
n_frames = 400
cam_sensors = ['CAM_FRONT', 'CAM_INFRASTRUCTURE_001', 'CAM_INFRASTRUCTURE_002', 'CAM_INFRASTRUCTURE_003']
lid_extent_infra = [[0, 60], [-60, 60], [-20, 20]]
lid_extent_ego = [[-60, 60], [-60, 60], [-20, 20]]

# prep video sizing
img_shape = (900, 1600, 3)
padding = 20
video_along_height = False
if video_along_height:
    width = img_shape[1]*2 + padding
    height = img_shape[0]*len(cam_sensors) + padding*(len(cam_sensors)-1)
else:
    height = img_shape[0]*2 + padding
    width = img_shape[1]*len(cam_sensors) + padding*(len(cam_sensors)-1)

fps = 20
writer = cv2.VideoWriter(
    "carla-collaborative-video-with-lidar.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
try:
    n_frames_max = min(n_frames, len(CSD.get_frames(sensor="CAM_FRONT"))-i_frame_start-n_safety_frames)
    for frame_idx in tqdm(range(i_frame_start, i_frame_start+n_frames_max, 1)):
        img_muxed = np.zeros((height, width, img_shape[2]), dtype=np.uint8)
        for i, cam_sensor in enumerate(cam_sensors):
            try:
                frame = CSD.get_frames(sensor=cam_sensor)[frame_idx]
            except IndexError:
                continue

            # Get sensor data
            lid_sensor = "LIDAR_TOP" if cam_sensor == "CAM_FRONT" else cam_sensor.replace("CAM", "LIDAR")
            img = CSD.get_image(frame, cam_sensor)
            pc = CSD.get_lidar(frame, lid_sensor)
            ego = CSD.get_ego(frame)
            
            # Get "local" objects and validate same|
            objects_cam = CSD.get_objects(frame=frame, sensor=cam_sensor, max_occ=avstack.environment.objects.Occlusion.MOST)
            objects_lid = CSD.get_objects(frame=frame, sensor=lid_sensor, max_occ=avstack.environment.objects.Occlusion.MOST)
    
            # Determine colors of objects
            box_colors_cam = []
            for obj in objects_cam:
                if obj.ID == ego.ID:
                    box_colors_cam.append('blue')
                else:
                    box_colors_cam.append('green')
            box_colors_lid = []
            for obj in objects_lid:
                if obj.ID == ego.ID:
                    box_colors_lid.append('blue')
                else:
                    box_colors_lid.append('green')
                    
            # get the camera visualization
            img_cam_out = avapi.visualize.snapshot.show_image_with_boxes(
                img=img,
                boxes=objects_cam,
                box_colors=box_colors_cam,
                inline=True,
                return_images=True,
                show=False
            )
    
            # get the BEV lidar visualization
            lid_extent = lid_extent_infra if "infra" in lid_sensor.lower() else lid_extent_ego
            img_lid_out = avapi.visualize.snapshot.show_lidar_bev_with_boxes(
                point_cloud=pc,
                boxes=objects_lid,
                box_colors=box_colors_lid,
                extent=lid_extent,
                bev_size=[img.shape[1], img.shape[0]],
                show=False,
                return_image=True,
            )
    
            # store the results
            if video_along_height:
                idx_store = img_shape[0]*i + padding*i
                img_muxed[idx_store:idx_store+img_shape[0], :img_shape[1],         ::-1] = img_cam_out
                img_muxed[idx_store:idx_store+img_shape[0], padding+img_shape[1]:, ::-1] = img_lid_out
            else:
                idx_store = img_shape[1]*i + padding*i
                img_muxed[:img_shape[0],         idx_store:idx_store+img_shape[1], ::-1] = img_cam_out
                img_muxed[padding+img_shape[0]:, idx_store:idx_store+img_shape[1], ::-1] = img_lid_out
    
        writer.write(img_muxed)
finally:
    writer.release()
    print('done making video.')